In [1]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datetime import datetime


In [3]:
data = pd.read_csv('../dataset/log_data2.csv')

In [4]:
# Convert timestamp to datetime objects

data['timestamp'] = pd.to_datetime(data['timestamp'])


In [5]:
# Calculate time difference in seconds from the minimum timestamp

min_timestamp = data['timestamp'].min()
data['time_diff'] = (data['timestamp'] - min_timestamp).dt.total_seconds()

In [6]:
# Select relevant features for anomaly detection
# In this case, we'll use time_diff, activity, and username
features = data[['time_diff', 'activity', 'username']]

In [7]:
# Convert categorical features to numerical using label encoding
label_encoders = {}
for col in ['activity', 'username']:
    label_encoders[col] = LabelEncoder()
    features[col] = label_encoders[col].fit_transform(features[col])

C:\Users\arunh\AppData\Local\Temp\ipykernel_16452\1446478807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[col] = label_encoders[col].fit_transform(features[col])
C:\Users\arunh\AppData\Local\Temp\ipykernel_16452\1446478807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[col] = label_encoders[col].fit_transform(features[col])


In [8]:
# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [9]:
# Create and train the Isolation Forest model
model = IsolationForest(contamination=0.05)  # Adjust contamination based on your data
model.fit(scaled_features)

IsolationForest(contamination=0.05)

In [10]:
# Predict anomalies (-1) and normal instances (1)
predictions = model.predict(scaled_features)

In [11]:
# Add the predictions back to the original dataset
data['anomaly_prediction'] = predictions

In [16]:
# Print the instances predicted as anomalies
anomalies = data[data['anomaly_prediction'] == -1]
print("Anomalies:")
print(anomalies)


Anomalies:
                timestamp      activity          username       source_ip  \
84    2023-01-05 03:42:18  AccessDenied  hernandezdarlene  113.116.46.121   
148   2023-01-02 23:12:33  AccessDenied     robertgilbert    49.188.43.93   
150   2023-08-04 19:46:34   ProductView      stevencooper   78.113.251.85   
151   2023-07-20 09:13:13   ProductView     alvarezaustin     98.23.86.50   
174   2023-01-08 17:19:24   ProductView          akennedy  167.82.195.116   
...                   ...           ...               ...             ...   
99624 2023-01-29 14:14:33   ProductView            beth44  161.36.103.142   
99653 2023-01-03 08:25:04   ProductView          fharding    100.61.92.13   
99695 2023-08-07 18:59:02   ProductView       kathyweaver  147.100.167.37   
99712 2023-01-01 01:14:24  AccessDenied        briandavis    85.43.55.118   
99765 2023-01-01 21:38:55   ProductView          joshua44   170.25.230.13   

        action   result   time_diff  anomaly_prediction  
84    